In [1]:
# import package
# installed via pip
from emtracks.particle import trajectory_solver # main solver object
from emtracks.conversions import one_gev_c2_to_kg # conversion for q factor (transverse momentum estimate)
from emtracks.tools import *#InitConds # initial conditions namedtuple
from emtracks.mapinterp import get_df_interp_func  # factory function for creating Mu2e DS interpolation function
from emtracks.Bdist import get_B_df_distorted
from emtracks.interpolations import *
import matplotlib.animation as animation
import numpy as np
from scipy.constants import c, elementary_charge
import pandas as pd
import pickle as pkl
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import math
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = [24,16] # bigger figures
from matplotlib import style
style.use('fivethirtyeight')
import os

testdir = "/home/darren/Desktop/plots/"
datadir = "/home/shared_data/"
plotdir = datadir+"plots/randomphi/"
mapdir = datadir+"Bmaps/"
date = "/7-9/"
newdir = datadir+'test4/'
maomapdir = '/home/shared_data/Bmaps/Mau10/combined/'


%matplotlib notebook


ModuleNotFoundError: No module named 'emtracks.Bdist'

In [7]:
files = sorted(os.listdir(maomapdir))
files[0]


'Mau10_1.00xDS_0.00xPS-TS_DSMap.p'

In [13]:
with open(maomapdir+'/'+files[0], 'rb') as pickle_file:
    new_data = pkl.load(pickle_file)

ValueError: unsupported pickle protocol: 5